# Day 7: Amplification Circuit
Based on the navigational maps, you're going to need to send more power to your ship's thrusters to reach Santa in time. To do this, you'll need to configure a series of amplifiers already installed on the ship.

There are five amplifiers connected in series; each one receives an input signal and produces an output signal. They are connected such that the first amplifier's output leads to the second amplifier's input, the second amplifier's output leads to the third amplifier's input, and so on. The first amplifier's input value is 0, and the last amplifier's output leads to your ship's thrusters.

```
    O-------O  O-------O  O-------O  O-------O  O-------O
0 ->| Amp A |->| Amp B |->| Amp C |->| Amp D |->| Amp E |-> (to thrusters)
    O-------O  O-------O  O-------O  O-------O  O-------O
    
```
The Elves have sent you some Amplifier Controller Software (your puzzle input), a program that should run on your existing Intcode computer. Each amplifier will need to run a copy of the program.

When a copy of the program starts running on an amplifier, it will first use an input instruction to ask the amplifier for its current phase setting (an integer from 0 to 4). Each phase setting is used exactly once, but the Elves can't remember which amplifier needs which phase setting.

The program will then call another input instruction to get the amplifier's input signal, compute the correct output signal, and supply it back to the amplifier with an output instruction. (If the amplifier has not yet received an input signal, it waits until one arrives.)

Your job is to find the largest output signal that can be sent to the thrusters by trying every possible combination of phase settings on the amplifiers. Make sure that memory is not shared or reused between copies of the program.

For example, suppose you want to try the phase setting sequence 3,1,2,4,0, which would mean setting amplifier A to phase setting 3, amplifier B to setting 1, C to 2, D to 4, and E to 0. Then, you could determine the output signal that gets sent from amplifier E to the thrusters with the following steps:

Start the copy of the amplifier controller software that will run on amplifier A. At its first input instruction, provide it the amplifier's phase setting, 3. At its second input instruction, provide it the input signal, 0. After some calculations, it will use an output instruction to indicate the amplifier's output signal.
Start the software for amplifier B. Provide it the phase setting (1) and then whatever output signal was produced from amplifier A. It will then produce a new output signal destined for amplifier C.
Start the software for amplifier C, provide the phase setting (2) and the value from amplifier B, then collect its output signal.

- Run amplifier D's software, provide the phase setting (4) and input value, and collect its output signal.
- Run amplifier E's software, provide the phase setting (0) and input value, and collect its output signal.

The final output signal from amplifier E would be sent to the thrusters. However, this phase setting sequence may not have been the best one; another sequence might have sent a higher signal to the thrusters.

In [1]:
from tools import get_data

In [2]:
data = get_data(7)

In [3]:
data

'3,8,1001,8,10,8,105,1,0,0,21,38,59,84,93,110,191,272,353,434,99999,3,9,101,5,9,9,1002,9,5,9,101,5,9,9,4,9,99,3,9,1001,9,3,9,1002,9,2,9,101,4,9,9,1002,9,4,9,4,9,99,3,9,102,5,9,9,1001,9,4,9,1002,9,2,9,1001,9,5,9,102,4,9,9,4,9,99,3,9,1002,9,2,9,4,9,99,3,9,1002,9,5,9,101,4,9,9,102,2,9,9,4,9,99,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,99,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,2,9,4,9,99,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,1,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,99,3,9,1001,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,

In [4]:
from intcode import *

In [95]:
class InterpreterContext:
    def __init__(self, tape, env):
        self.tape = tape
        self.env = env
        
class CallContext:
    def __init__(self, interpreter_ctx, argspec, op, call_mode, inputs):
        self.tape = interpreter_ctx.tape
        self.env = interpreter_ctx.env
        self.mode = call_mode
        self.inputs = inputs
        self.argspec = argspec
        self.op = op
        
    def __getitem__(self, index):
        argspec = self.argspec[index]
        literal = self.inputs[index]
        if argspec is L_OR_A:
            return literal if self.mode[index] else self.tape[literal]
        elif argspec is ADDRESS:
            return self.tape[literal]
        else:
            return literal
    
    def __len__(self):
        return len(self.inputs)
    
    def arg_repr(self, index):
        argspec = self.argspec[index]
        literal = self.inputs[index]
        if argspec is L_OR_A:
            return f'{literal}L' if self.mode[index] else f'[{literal}](->{self.tape[literal]})'
        elif argspec is ADDRESS:
            return f'[{literal}](->{self.tape[literal]})'
        else:
            return f'{literal}L'
    
    def __repr__(self):
        return f'CallContext: {self.op.__name__} {" ".join(self.arg_repr(i) for i in range(len(self)))}'

In [99]:
_ALL_OPS = []

class IntcodeInterpreter:
    def __init__(self):
        self.registered_ops = {}
        self.register_default_ops(_ALL_OPS)
        
    def evaluate(self, tape, env=StdioEnv(), debug=False):
        tape = list(tape)
        ctx = InterpreterContext(tape, env)
        index = 0
        ctx.env.reset()
        while tape[index] != HALT:
            opcode = tape[index]
            opcode_base = opcode % 100
            op, argspec = self.registered_ops[opcode_base]
            argcount = len(argspec)
            mode_v = (opcode - (opcode % 100)) // 100
            mode_s = str(mode_v)
            padded_mode_s = '0' * (argcount - len(mode_s)) + mode_s
            mode = [int(c) for c in reversed(padded_mode_s)]
            next_index = index + argcount + 1
            call_ctx = CallContext(ctx, argspec, op, mode, tape[index+1:next_index])
            if debug:
                print(call_ctx)
            ret = op(call_ctx)
            index = next_index if ret is None else ret
        return ctx
    
    def register_default_ops(self, all_ops):
        for fn, opcode, argspec, is_official in all_ops:
            if is_official:
                self.registered_ops[opcode] = (fn, argspec)
                
    
def register_op(opcode: int, args, is_official=True):
    def decorator(fn):
        _ALL_OPS.append((fn, opcode, args, is_official))
        return fn
    return decorator

LITERAL, ADDRESS, L_OR_A = object(), object(), object()

@register_op(opcode=1, args=[L_OR_A, L_OR_A, LITERAL])
def add(ctx):
    a, b, oi = ctx
    ctx.tape[oi] = a + b

@register_op(opcode=2, args=[L_OR_A, L_OR_A, LITERAL])
def multiply(ctx):
    a, b, oi = ctx
    ctx.tape[oi] = a * b

@register_op(opcode=3, args=[LITERAL])
def _input(ctx):
    (oi,) = ctx
    ctx.tape[oi] = int(ctx.env.input())

@register_op(opcode=4, args=[L_OR_A])
def output(ctx):
    (v,) = ctx
    ctx.env.output(v)

@register_op(opcode=5, args=[L_OR_A, L_OR_A])
def jump_if_true(ctx):
    v, ji = ctx
    if v:
        return ji

@register_op(opcode=6, args=[L_OR_A, L_OR_A])
def jump_if_false(ctx):
    v, ji = ctx
    if not v:
        return ji

@register_op(opcode=7, args=[L_OR_A, L_OR_A, LITERAL])
def less_than(ctx):
    a, b, oi = ctx
    ctx.tape[oi] = int(a < b)

@register_op(opcode=8, args=[L_OR_A, L_OR_A, LITERAL])
def equals(ctx):
    a, b, oi = ctx
    ctx.tape[oi] = int(a == b)

In [103]:
class StdioEnv:
    outputs = ()

    def reset(self):
        pass

    def input(self):
        return int(input('Requesting program input> '))

    def output(self, value):
        print("Output:", value)


class ProgrammaticEnv:
    def __init__(self, inputs):
        self.inputs = list(inputs)
        self.reset()

    def reset(self):
        self._iter = iter(self.inputs)
        self.outputs = []

    def input(self):
        return int(next(self._iter))

    def output(self, value):
        self.outputs.append(value)
        
    def __repr__(self):
        return f"ProgrammaticEnv(outputs={self.outputs})"

In [100]:
interpreter = IntcodeInterpreter()

In [101]:
interpreter.evaluate(make_tape(data), debug=True)

CallContext: _input 8L
Requesting program input> 4
CallContext: add [8](->4) 10L 8L
CallContext: jump_if_true 1L [14](->93)
CallContext: _input 9L
Requesting program input> 3
CallContext: multiply [9](->3) 5L 9L
CallContext: add 4L [9](->15) 9L
CallContext: multiply 2L [9](->19) 9L
CallContext: output [9](->38)
Output: 38


In [105]:
interpreter.evaluate(make_tape(data), ProgrammaticEnv([4, 3])).env.outputs

[38]

Here are some example programs:


Max thruster signal `43210` (from phase setting sequence 4,3,2,1,0):
```
3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0
```

Max thruster signal `54321` (from phase setting sequence 0,1,2,3,4):
```
3,23,3,24,1002,24,10,24,1002,23,-1,23,
101,5,23,23,1,24,23,23,4,23,99,0,0
```

Max thruster signal `65210` (from phase setting sequence 1,0,4,3,2):
```
3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,
1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0
```

In [106]:
interpreter.evaluate(make_tape('3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0')).env.outputs

Requesting program input> 4
Requesting program input> 3
Output: 34


()

In [108]:
interpreter.evaluate(make_tape('3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0')).env.outputs

Requesting program input> 0
Requesting program input> 1
Output: 15


()

In [116]:
import itertools

def phase_combinations():
    return itertools.permutations(range(5))

def compute_output_thrust_signal(amplifier_tape, phases):
    current_signal = 0
    for amplifier, phase in enumerate(phases):
        result = interpreter.evaluate(amplifier_tape, ProgrammaticEnv([phase, current_signal]))
        output_signal = result.env.outputs[0]
        current_signal = output_signal
    return current_signal

In [110]:
compute_output_thrust_signal(make_tape('3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0'), [4, 3, 2, 1, 0])

43210

In [112]:
compute_output_thrust_signal(
    make_tape('3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0'),
    [0, 1, 2, 3, 4],
)

54321

In [113]:
def maximum_output_thrust_signal(amplifier_tape):
    best_phase = max(phase_combinations(), key=lambda phase: compute_output_thrust_signal(amplifier_tape, phase))
    return best_phase, compute_output_thrust_signal(amplifier_tape, best_phase)

In [118]:
maximum_output_thrust_signal(make_tape('3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0'))

((0, 1, 2, 3, 4), 54321)

In [119]:
maximum_output_thrust_signal(make_tape(data))

((0, 1, 2, 4, 3), 225056)

### Part Two

It's no good - in this configuration, the amplifiers can't generate a large enough output signal to produce the thrust you'll need. The Elves quickly talk you through rewiring the amplifiers into a feedback loop:

```
      O-------O  O-------O  O-------O  O-------O  O-------O
0 -+->| Amp A |->| Amp B |->| Amp C |->| Amp D |->| Amp E |-.
   |  O-------O  O-------O  O-------O  O-------O  O-------O |
   |                                                        |
   '--------------------------------------------------------+
                                                            |
                                                            v
                                                     (to thrusters)
```

Most of the amplifiers are connected as they were before; amplifier A's output is connected to amplifier B's input, and so on. However, the output from amplifier E is now connected into amplifier A's input. This creates the feedback loop: the signal will be sent through the amplifiers many times.

In feedback loop mode, the amplifiers need totally different phase settings: integers from 5 to 9, again each used exactly once. These settings will cause the Amplifier Controller Software to repeatedly take input and produce output many times before halting. Provide each amplifier its phase setting at its first input instruction; all further input/output instructions are for signals.

Don't restart the Amplifier Controller Software on any amplifier during this process. Each one should continue receiving and sending signals until it halts.

All signals sent or received in this process will be between pairs of amplifiers except the very first signal and the very last signal. To start the process, a 0 signal is sent to amplifier A's input exactly once.

Eventually, the software on the amplifiers will halt after they have processed the final loop. When this happens, the last output signal from amplifier E is sent to the thrusters. Your job is to find the largest output signal that can be sent to the thrusters using the new phase settings and feedback loop arrangement.

In [156]:
_ALL_OPS = []
YIELD = object()

class InterpreterContext:
    def __init__(self, tape, env, index=0, state=None, trampoline=False, debug=False):
        self.tape = tape
        self.env = env
        self.state = state
        self.debug = debug
        self.trampoline = trampoline
        self.index = index
        
    def __repr__(self):
        return f'''
            InterpreterContext(
                index={self.index},
                state={self.state},
                tape={self.tape},
            )
            '''

class IntcodeInterpreter:
    def __init__(self):
        self.registered_ops = {}
        self.register_default_ops(_ALL_OPS)
        
    def evaluate(self, tape, env=StdioEnv(), trampoline=False, debug=False):
        tape = list(tape)
        ctx = InterpreterContext(tape, env)
        ctx.env.reset()
        return self.evaluate_from_context(ctx)
    
    def evaluate_from_context(self, ctx):
        ctx.state = 'RUNNING'
        tape = ctx.tape
        debug = ctx.debug
        index = ctx.index
        while tape[index] != HALT:
            opcode = tape[index]
            opcode_base = opcode % 100
            op, argspec = self.registered_ops[opcode_base]
            argcount = len(argspec)
            mode_v = (opcode - (opcode % 100)) // 100
            mode_s = str(mode_v)
            padded_mode_s = '0' * (argcount - len(mode_s)) + mode_s
            mode = [int(c) for c in reversed(padded_mode_s)]
            next_index = index + argcount + 1
            call_ctx = CallContext(ctx, argspec, op, mode, tape[index+1:next_index])
            if debug:
                print(call_ctx)
            ret = op(call_ctx)
            index = next_index if ret is None else ret
            ctx.index = index
            
            # Handle trampoline YIELD cases
            if op is output and ctx.trampoline:
                ctx.state = YIELD
                return ctx
        
        ctx.state = HALT
        return ctx
    
    def register_default_ops(self, all_ops):
        for fn, opcode, argspec, is_official in all_ops:
            if is_official:
                self.registered_ops[opcode] = (fn, argspec)
                
    
def register_op(opcode: int, args, is_official=True):
    def decorator(fn):
        _ALL_OPS.append((fn, opcode, args, is_official))
        return fn
    return decorator

LITERAL, ADDRESS, L_OR_A = object(), object(), object()

@register_op(opcode=1, args=[L_OR_A, L_OR_A, LITERAL])
def add(ctx):
    a, b, oi = ctx
    ctx.tape[oi] = a + b

@register_op(opcode=2, args=[L_OR_A, L_OR_A, LITERAL])
def multiply(ctx):
    a, b, oi = ctx
    ctx.tape[oi] = a * b

@register_op(opcode=3, args=[LITERAL])
def _input(ctx):
    (oi,) = ctx
    ctx.tape[oi] = int(ctx.env.input())

@register_op(opcode=4, args=[L_OR_A])
def output(ctx):
    (v,) = ctx
    ctx.env.output(v)

@register_op(opcode=5, args=[L_OR_A, L_OR_A])
def jump_if_true(ctx):
    v, ji = ctx
    if v:
        return ji

@register_op(opcode=6, args=[L_OR_A, L_OR_A])
def jump_if_false(ctx):
    v, ji = ctx
    if not v:
        return ji

@register_op(opcode=7, args=[L_OR_A, L_OR_A, LITERAL])
def less_than(ctx):
    a, b, oi = ctx
    ctx.tape[oi] = int(a < b)

@register_op(opcode=8, args=[L_OR_A, L_OR_A, LITERAL])
def equals(ctx):
    a, b, oi = ctx
    ctx.tape[oi] = int(a == b)

In [164]:
def compute_feedback_thrust_signal(interpreter, amplifier_tape, phases):
    current_signal = 0
    contexts = [InterpreterContext(list(amplifier_tape), env=None, trampoline=True) for _ in phases]
    outputs = []
    started = set()
    for ctx, phase in itertools.cycle(zip(contexts, phases)):
        if phase not in started:
            inputs = [phase, current_signal]
            started.add(phase)
            ctx.env = ProgrammaticEnv(inputs)
        else:
            ctx.env = ProgrammaticEnv([current_signal])
        result = interpreter.evaluate_from_context(ctx)
        if result.state is HALT:
            break
        elif result.state is YIELD:
            output_signal = result.env.outputs[0]
            current_signal = output_signal
            outputs.append(output_signal)
    # We care about the last output given by the last phase, in case that isn't the one
    # that halts
    return outputs[-1 - (len(outputs) % len(phases))]

Here are some example programs:

Max thruster signal `139629729` (from phase setting sequence 9,8,7,6,5):
```
3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,
27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5
```

Max thruster signal `18216` (from phase setting sequence 9,7,8,5,6):

```
3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,
-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,
53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10
```

Try every combination of the new phase settings on the amplifier feedback loop. What is the highest signal that can be sent to the thrusters?

In [165]:
compute_feedback_thrust_signal(
    IntcodeInterpreter(),
    make_tape('3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5'),
    [9, 8, 7, 6, 5],
)

139629729

In [166]:
compute_feedback_thrust_signal(
    IntcodeInterpreter(),
    make_tape('3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10'),
    [9, 7, 8, 5, 6],
)

18216

In [174]:
def maximum_feedback_output_thrust_signal(amplifier_tape):
    possible_phases = itertools.permutations(range(5, 10))
    best_phase = max(possible_phases, key=lambda phase: compute_feedback_thrust_signal(IntcodeInterpreter(), amplifier_tape, phase))
    return best_phase, compute_feedback_thrust_signal(IntcodeInterpreter(), amplifier_tape, best_phase)

In [175]:
maximum_feedback_output_thrust_signal(make_tape('3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5'))

((9, 8, 7, 6, 5), 139629729)

In [176]:
maximum_feedback_output_thrust_signal(make_tape(data))

((8, 5, 9, 6, 7), 14260332)